## Changelog:

* v8 Checking code and revising the debug files. Ready for debugging 
* V9 Adding resampling
* V10 Adding tensorboard
* V11 Reducing LSTM layer to 1 in order to have better interpretation in tensorboard
* V12 adding actionId to data in order to debug datasets and step better + fixing the wrong column name for x_prepared + **fixing LABEL WRONG!!!!! ASSIGENMENT **
* V13 removing masking layer and padding with 0 or 9999
* V14 removing two outputs
* V15 returning the masking layer with mask value 99.
    - The result is that without adding masking layer the training accuracy does not increase at all and it remains the same on nearly 50
    - however the validation accuracy remains the same around 50 and does not increase while the validation loss is being increased as allways
* V16 Changing the optimizer to SGD
* V17 Changing the optimizer to rmsprop
* V18 Increasing batch size from 1 to higher and coming back the optimizer to Adam (both roc and accuracy on train was about .9 while the accuracy of validation was 0.5
* V19 Separating validation set from data and feed to the fit function using validation_data param
* V20 Adding another layer of 100 unit
* V21 Joining per stud features
* V22 removing the sampling
* V23 Add two output
* V24 Assumes that we are overfitting so we are going to:
    - Remove additional LSTM layer to simplify the model $\checkmark$
    - Reduce the number of units for LSTM layer $\checkmark$ (saw that finally validation loss starts to deacrease with 20 unit and 200 seq lenght)
    - Reduce the number of features as much as possible (should be done after dinormalizing the binary variables)
    - Reduce the number of seq length $\checkmark$
* V25 Excluding binary variables and some other proportional variables from the standardization step (it seems we are improving!)
* V26 Adding competition validation set for making prediction on un-labeled data
* Bringing resampling back to the game!
* V28 Adding Startify spliting
* V29 Inversing the sequence! (it make more sence to feed the old frames first and then go forward for newer frames)
* V30 A revolutionary update: removing the padding and updating the weights with variable lenght sequences!
* V31 Testing various model architecture and layers with different number of units
    - V31.1 Disabling resampling
* V32 Exclude students with large number of actions e.g. greater than 2000
* V33 Encode categorical features + **USING MinMaxScaler**
* V34 Removing outliers for MinMax feature scaling

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from keras.layers import LSTM, Dense, Dropout, SimpleRNN, GRU
from keras.models import Sequential
from keras.callbacks import TensorBoard, EarlyStopping
from keras.preprocessing import sequence
from keras import optimizers

from Vis import plot_loss, plot_roc, plot_accuracy
from Preprocessing import Preprocessing, Cols

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, MinMaxScaler

from matplotlib import pyplot as plt

from pandas import DataFrame, Series
import pandas as pd

import numpy as np

from math import sqrt
from itertools import chain

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# pandas.set_option('max_columns',10)

import os
import glob
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
# Loading datasets and labels
data_files = glob.glob(os.path.join("Dataset", "student_log_*.csv"))
raw_dataset = pd.concat((pd.read_csv(f, index_col=["ITEST_id"]) for f in data_files))

# dataset = raw_dataset.drop(Cols.excluded_cols + Cols.cat_cols, axis=1)
dataset = raw_dataset.drop(Cols.excluded_cols, axis=1)

labels = DataFrame.from_csv("Dataset/training_label.csv")
valid_test_label_dataset = DataFrame.from_csv("Dataset/validation_test_label.csv")

unlabels = valid_test_label_dataset.drop(list(labels.index.intersection(valid_test_label_dataset.index)))
labels_unlabels = pd.concat([labels,unlabels])

dwlu = dataset.join(labels_unlabels, how="inner") # dwlu = dataset_with_labels_unlabels

dwlu.index = pd.MultiIndex.from_arrays([dwlu.index, dwlu.actionId])

dwlu = dwlu.drop("actionId", axis =1)
dwlu = dwlu.sort_values("startTime")

In [3]:
# Converting category variables to dummy variables
cat_cols = ['skill', 'problemType', 'SY ASSISTments Usage', 'MCAS', 'SchoolId']

new_cols = [{cc: len(dwlu[cc].unique())} for cc in cat_cols]
print("New Columns:" , *new_cols)
dwlu = pd.get_dummies(dwlu, columns=cat_cols)

New Columns: {'skill': 93} {'problemType': 16} {'SY ASSISTments Usage': 2} {'MCAS': 51} {'SchoolId': 4}


In [4]:
# Excluding students with large number of actions (does not matter whether they are isSTEM=1 or not but does matter if they are isSTEM=NAN)
isLarge = (dwlu.groupby("ITEST_id").size() > 2000)
largeStuds_ids = isLarge[isLarge == True].index.values
largeStuds_ids_with_label = [l for l in largeStuds_ids if l not in unlabels.index.values]

print("%d students are removed!" % len(largeStuds_ids_with_label))
dwlu = dwlu.drop(largeStuds_ids_with_label, level=0)

# no unlabeled data should be removed
assert(len(dwlu[dwlu.isSTEM.isnull()].index.get_level_values(0).unique()) == len(unlabels))

6 students are removed!


In [5]:
# Resampling data in order to balance class labels
from sklearn.utils import resample

df_majority = dwlu[dwlu['isSTEM'] == 0]
df_minority = dwlu[dwlu['isSTEM'] == 1]
df_unlabeled = dwlu[dwlu['isSTEM'].isnull()]

minority_len = len(df_minority.index.get_level_values(0).unique())

majority_ids = df_majority.index.get_level_values(0).unique()
sample_majority_ids = resample(majority_ids , n_samples=minority_len, replace=False).values

sampled_df_majority = df_majority.sort_index(level=0).loc[sample_majority_ids, :]

dwlu = pd.concat([df_minority, sampled_df_majority, df_unlabeled])

In [6]:
# Listing all dummy variables
all_dummy_cols = [[col for col in dwlu.columns if cat+"_" in col] for cat in cat_cols ]
all_dummy_cols = list(chain.from_iterable(all_dummy_cols))

In [7]:
# Specifying columns that should be normalized
binary_cols = ['AveKnow', 'AveCarelessness', 'correct', 'original', 'hint', 'scaffold', 'bottomHint', 'frIsHelpRequest', 'stlHintUsed', 'frWorkingInSchool',
               'responseIsFillIn', 'responseIsChosen', 'endsWithScaffolding', 'endsWithAutoScaffolding', 'frIsHelpRequestScaffolding', 'timeGreater5Secprev2wrong', 'helpAccessUnder2Sec', 'timeGreater10SecAndNextActionRight', 'timeOver80', 'manywrong']
res_cols = ['RES_BORED', 'RES_CONCENTRATING', 'RES_CONFUSED', 'RES_FRUSTRATED', 'RES_OFFTASK', 'RES_GAMING']
should_not_normalize_cols = ['isSTEM'] + res_cols + binary_cols + all_dummy_cols
# also for 'AveCorrect', 'AveResBored', 'AveResEngcon', 'AveResConf', 'AveResFrust', 'AveResOfftask', 'AveResGaming', 'Ln-1', 'Ln', 

should_not_normalized = dwlu[should_not_normalize_cols]
should_normalized = dwlu.drop(should_not_normalize_cols, axis =1)

In [9]:
# Scaling necessary columns
scaler = MinMaxScaler(feature_range=(-1,1))
scaled_data = scaler.fit_transform(should_normalized)
scaled_data = DataFrame(scaled_data, index=should_normalized.index, columns=should_normalized.columns)
scaled_dwlu = scaled_data.join(should_not_normalized)

In [10]:
# Removing some features to avoid overfitting
unimportant_features = ["problemId", 'startTime', 'endTime', 'assignmentId', 'assistmentId'] + ['AveCarelessness', 'AveKnow', 'NumActions', 'RES_BORED',
                       'RES_CONCENTRATING', 'RES_CONFUSED', 'RES_FRUSTRATED', 'RES_GAMING',
                       'RES_OFFTASK', 'manywrong', 'timeOver80']
Cols.paper_suggested_cols
scaled_dwlu = scaled_dwlu.drop(unimportant_features, axis=1)

In [11]:
# Building learning datasets/labels and competition preditcion dataset
x_competition = scaled_dwlu.loc[unlabels.index.values, :].drop("isSTEM", axis=1)
x = scaled_dwlu.loc[labels.index.values, :].drop("isSTEM", axis=1)
y = scaled_dwlu.loc[labels.index.values, :][['isSTEM']].reset_index(level=1, drop=True)
y = y[~y.index.duplicated()]

In [12]:
# Converting X from a 2D dataframe to a 3D list of varibale-lenght sequecens
x = np.array([stud_seq.values for _, stud_seq in x.groupby("ITEST_id")])
y = y.values

In [13]:
# Splitting train/test data 
# TODO test Startify
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.85)

In [14]:
# Encoding train/test labels
enc = OneHotEncoder()
y_train = enc.fit_transform(y_train.reshape(-1,1)).toarray()
y_test = enc.fit_transform(y_test.reshape(-1,1)).toarray()

## Model

In [15]:
# Defining model configs.
batch_size = 1 # sequences are independent and the network weights should be updated after each sequence
feature_size = x_train[0].shape[1] # num. of features extracted from first training sample
timestep_size = None # here we are using variable-length sequences so there is no fixed timestep size
nb_epoch = 10

In [16]:
# Creating necessary callbacks for early stopping and Tensorboard visualization
tensorboard_callback = TensorBoard(log_dir='./logs', histogram_freq=1, batch_size=batch_size, write_graph=True, write_grads=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0.005, patience=5, verbose=10, mode='min')

In [ ]:
# Defining a variable length model!
def create_LSTM_model(feature_size):
    model = Sequential()
    model.add(GRU(200, input_shape=(None, feature_size)))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
    
    return model

In [ ]:
# Creating and fitting model
model = create_LSTM_model(feature_size)
epoch_histories = []
for i in range(nb_epoch):
    print("epoch: {}".format(i))
    train_history = []
    for seq, label in zip(x_train, y_train):
        loss = model.fit(np.array([seq]), label.reshape(1,2), epochs=1, batch_size=batch_size)
        train_history.append(loss.history)
    
    val_history = []
    for seq_val, label_val in zip(x_test, y_test):
        val_loss = model.evaluate(np.array([seq_val]),label_val.reshape(1,2), batch_size=batch_size)
        val_history.append(val_loss)
        print("val_loss for each sample at the end of epoch: {}".format(val_loss))
    epoch_histories.append({"train_history": train_history, "val_history": val_history})

epoch: 0
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 0.7271 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 117ms/step - loss: 0.2733 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 446ms/step - loss: 0.2942 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 127ms/step - loss: 2.6346 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 462ms/step - loss: 2.0815 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 99ms/step - loss: 2.0968 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 191ms/step - loss: 0.1590 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 314ms/step - loss: 0.2645 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 1s/step - loss: 1.1215 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 77ms/step - loss: 1.0088 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 46m

Epoch 1/1
1/1 [==============================] - 0s 97ms/step - loss: 0.3055 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 443ms/step - loss: 1.1184 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 316ms/step - loss: 0.4390 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 237ms/step - loss: 0.3868 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 422ms/step - loss: 0.4265 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 85ms/step - loss: 0.5432 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 486ms/step - loss: 1.1882 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 1s 830ms/step - loss: 1.0542 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 1s 924ms/step - loss: 0.9130 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 230ms/step - loss: 0.5565 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 132ms/step - l

Epoch 1/1
1/1 [==============================] - 0s 160ms/step - loss: 0.4098 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 310ms/step - loss: 1.2077 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 190ms/step - loss: 0.3150 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 561ms/step - loss: 0.2731 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 364ms/step - loss: 0.4906 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 574ms/step - loss: 1.2886 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 61ms/step - loss: 0.6113 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 506ms/step - loss: 0.2777 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 62ms/step - loss: 0.1529 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 254ms/step - loss: 0.7771 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 393ms/step - loss:

Epoch 1/1
1/1 [==============================] - 0s 194ms/step - loss: 0.4896 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 165ms/step - loss: 1.2311 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 71ms/step - loss: 0.2141 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 41ms/step - loss: 0.8444 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 191ms/step - loss: 0.6109 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 81ms/step - loss: 0.3331 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 637ms/step - loss: 0.7823 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 177ms/step - loss: 0.6987 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 288ms/step - loss: 0.5226 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 215ms/step - loss: 0.3704 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 595ms/step - lo

Epoch 1/1
1/1 [==============================] - 1s 614ms/step - loss: 0.6280 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 307ms/step - loss: 0.2686 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 65ms/step - loss: 0.3772 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 248ms/step - loss: 0.1779 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 45ms/step - loss: 0.1528 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 171ms/step - loss: 0.4869 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 105ms/step - loss: 1.2188 - acc: 0.0000e+00
Epoch 1/1
1/1 [==============================] - 0s 235ms/step - loss: 0.2231 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 319ms/step - loss: 0.3978 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 0s 68ms/step - loss: 0.0553 - acc: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 951ms/step - loss: 0.3331 -

In [ ]:
def accumulate_loss_acc():
    for ep_hist in epoch_histories:
        
        tr_hist_list = ep_hist.get("train_history")
        train_acc = np.asscalar(np.mean(np.array([sample_history.get("acc") for sample_history in tr_hist_list]), axis=0))
        train_loss = np.asscalar(np.mean(np.array([sample_history.get("loss") for sample_history in tr_hist_list]), axis=0))
        
        val_hist_list = ep_hist.get("val_history")
        
        val_loss_acc = np.mean(np.array(val_hist_list), axis=0)
        
        yield np.hstack((np.array([train_loss, train_acc]), val_loss_acc))

In [ ]:
train_val_loss_acc = np.array([s for s in accumulate_loss_acc()])

In [ ]:
fig, (loss, acc) = plt.subplots(1,2)
fig.set_size_inches(15,5)

loss.plot(np.arange(nb_epoch), train_val_loss_acc[:,0], label="train_loss")
loss.plot(np.arange(nb_epoch), train_val_loss_acc[:,2], label="val_loss")
acc.plot(np.arange(nb_epoch), train_val_loss_acc[:,1], label="train_acc")
acc.plot(np.arange(nb_epoch), train_val_loss_acc[:,3], label="val_acc")
loss.legend()
acc.legend()

In [ ]:
# do prediction
predictions = []
for seq_test, label_test in zip(x_test, y_test):
    pred = model.predict(np.array([seq_test]), batch_size=batch_size)
    predictions.append(pred)

In [ ]:
y_true = np.argmax(y_test, axis=1)
y_pred = np.argmax(predictions, axis=2)

In [ ]:
# calculating scores
roc_score = roc_auc_score(y_true, y_pred)
# roc_score = roc_auc_score(np.argmax(y_test,axis=1), np.argmax(y_pred,axis=1))

mse_score = mean_squared_error(y_true, y_pred)

#printing and plotting model and score information
# plot_loss(history)
# plot_roc(y_test, y_pred)
plot_roc(y_true, y_pred)
# plot_accuracy(history)

print(model.summary())
print("Test ROC Score: %f" % roc_score)
print("Test RMSE Score: %f" % sqrt(mse_score))
print("Final Competition Score: %f" % (1 - sqrt(mse_score) + roc_score))

In [ ]:
# # do prediction
# predictions = []
# for seq_test, label_test in zip(x_test, y_test):
#     pred = model.predict(np.array([seq_test]), batch_size=batch_size)
#     predictions.append(pred)

In [ ]:
# y_pred_competition = model.predict(x_competition_arr, batch_size=batch_size)

# result_index = x_competition.reset_index(level=1, drop=True).index.unique()

# argmax_preds = [np.argmax(predicted_label) for predicted_label in y_pred_competition]

# result_df = DataFrame(argmax_preds, index=pd.Index(result_index, name='ITEST_id'), columns=['isSTEM'])

# final_output = pd.concat([result_df, label_dataset.loc[shared_ids_with_train.values]]).sort_index()
# final_output.to_csv("submition_1_{}.csv".format(theNotebook))
# final_output.isSTEM.value_counts()